<a href="https://colab.research.google.com/github/marcomachado/teo-me-why-web-scrapping/blob/main/resident_evil_collect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests

!pip install beautifulsoup4
from bs4 import BeautifulSoup

!pip install tqdm
from tqdm import tqdm

import pandas as pd
import os
os.makedirs('dados')

In [ ]:
# google curl to python
# https://curlconverter.com/

def get_content(url):
  headers = {
      'authority': 'www.residentevildatabase.com',
      'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
      'accept-language': 'en-US,en;q=0.9',
      'cache-control': 'max-age=0',
      # 'cookie': '_gid=GA1.2.1217994358.1711368764; __gads=ID=c5c850d56fc1456d:T=1711368764:RT=1711370017:S=ALNI_MaRDUZPK7sq-7ZUrSLdJo9nipwLBg; __gpi=UID=00000dd575eed7dc:T=1711368764:RT=1711370017:S=ALNI_MaH9Do1WUwlV97HnV7lVMlmcSVhJQ; __eoi=ID=35dfcdb5a17ef10c:T=1711368764:RT=1711370017:S=AA-Afja9CtdDDzUW6f_AurIIKXS_; _ga=GA1.2.271214056.1711368763; clever-last-tracker-71584=3; clever-counter-71584=0-3; _ga_DJLCSW50SC=GS1.1.1711368762.1.1.1711370023.52.0.0; _ga_D6NF5QC4QT=GS1.1.1711368764.1.1.1711370023.52.0.0; FCNEC=%5B%5B%22AKsRol_e7EhI1k7oT0HB_eXYiJx1Zhqz0BwTXvTudtnkvbz9tnZ6-nCOwyvTmXPcy90qZHA0BZfElrS1Q_bVz4WHEmoyYE8r29kvCFXEiJ1h2CspotpxXDd2lgVbKx1ic2Lb31dn4yKLlnx9c4bvIZOfgZibwDyZgA%3D%3D%22%5D%5D',
      'referer': 'https://www.residentevildatabase.com/personagens/',
      'sec-ch-ua': '"Chromium";v="122", "Not(A:Brand";v="24", "Google Chrome";v="122"',
      'sec-ch-ua-mobile': '?0',
      'sec-ch-ua-platform': '"Linux"',
      'sec-fetch-dest': 'document',
      'sec-fetch-mode': 'navigate',
      'sec-fetch-site': 'same-origin',
      'sec-fetch-user': '?1',
      'upgrade-insecure-requests': '1',
      'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36',
  }
  response = requests.get(url, headers=headers)
  return response

In [ ]:
def get_basic_infos(soup):
  try:
    div_page = soup.find("div", class_='td-page-content')
    paragrafo = div_page.find_all("p")[1]
    ems = paragrafo.find_all("em")

    data = {}
    for i in ems:
      chave, valor = i.text.split(':')
      chave = chave.strip(' ')
      data[chave] = valor.strip(' ')

    return data
  except:
    print(f'Erro na página {soup.title.string} - OP get_basic_infos')
    return {}

In [ ]:
# Buscando aparições do personagem
# elemento na página próximo a lista: <h4>Aparições em títulos da série:</h4>
# usar find_next para pegar a lista
# localiza todos os li -> itens da lista
def get_aparicoes(soup):
  try:
    lis = soup.find("div", class_='td-page-content').\
      find('h4').\
      find_next().\
      find_all('li')

    aparicoes = [i.text for i in lis]
    return aparicoes
  except:
    print(f'Erro na página {soup.title.string} - OP get_aparicoes')
    return []

In [ ]:
def get_personagem_infos(url):
  resp = get_content(url)
  if resp.status_code != 200:
    print(f'Erro na URL {url}')
    print(f'Código de erro: {resp.status_code}')
    return {}
  else:
    soup = BeautifulSoup(resp.text)
    data = get_basic_infos(soup)
    data['Aparicoes'] = get_aparicoes(soup)
    return data

In [ ]:
def get_links_personagens():
  url_personagens = 'https://www.residentevildatabase.com/personagens/'
  resp = get_content(url_personagens)
  soup = BeautifulSoup(resp.text)
  links_ancoras = soup.find('div', class_='td-page-content').find_all('a')

  links = [i['href'] for i in links_ancoras]
  return links

In [ ]:
links = get_links_personagens()
data = []
for link in tqdm(links):
  infos = get_personagem_infos(link)
  infos['link'] = link
  nome = link.split('/')[-2].replace('-', ' ')
  infos['Nome'] = nome
  data.append(infos)


 10%|▉         | 16/164 [00:19<02:49,  1.15s/it]

Erro na página Ark Thompson | Resident Evil Database - OP get_basic_infos


 31%|███       | 51/164 [01:00<01:58,  1.05s/it]

Erro na URL https://www.residentevildatabase.com/doug-re5-desperate-escape/
Código de erro: 404


 56%|█████▌    | 92/164 [01:46<01:20,  1.12s/it]

Erro na página John | Resident Evil Database - OP get_basic_infos


 71%|███████   | 116/164 [02:16<00:58,  1.21s/it]

Erro na página Michael Warren | Resident Evil Database - OP get_basic_infos


100%|██████████| 164/164 [03:13<00:00,  1.18s/it]


In [ ]:
data

[{'Ano de nascimento': '1974 (24 anos em 1998)',
  'Tipo sanguíneo': 'AB',
  'Altura': 'Desconhecida.',
  'Peso': 'Desconhecido.',
  'Aparicoes': ['Biohazard / Resident Evil 2 (1998)',
   'Biohazard / Resident Evil 4 (2005)',
   'Biohazard / Resident Evil: The Umbrella Chronicles (2007)',
   'Biohazard / Resident Evil: The Darkside Chronicles (2009)',
   'Biohazard / Resident Evil: Operation Raccoon City (2012)',
   'Biohazard / Resident Evil: Damnation (2012)',
   'Biohazard / Resident Evil 6 (2012)',
   'Biohazard RE:2 / Resident Evil 2 (2019)',
   'Biohazard RE:4 / Resident Evil 4 (2023)'],
  'link': 'https://www.residentevildatabase.com/ada-wong/',
  'Nome': 'ada wong'},
 {'Ano de nascimento': 'Desconhecido.',
  'Tipo sanguíneo': 'Desconhecido.',
  'Altura': 'Desconhecida.',
  'Peso': 'Desconhecido.',
  'Aparicoes': ['Biohazard / Resident Evil 6 (2012)'],
  'link': 'https://www.residentevildatabase.com/adam-benford/',
  'Nome': 'adam benford'},
 {'Ano de nascimento': 'Desconhecido.

In [ ]:
df = pd.DataFrame(data)
df.head()

,Ano de nascimento,Tipo sanguíneo,Altura,Peso,Aparicoes,link,Nome,de nascimento
0,1974 (24 anos em 1998),AB,Desconhecida.,Desconhecido.,"[Biohazard / Resident Evil 2 (1998), Biohazard...",https://www.residentevildatabase.com/ada-wong/,ada wong,NaN
1,Desconhecido.,Desconhecido.,Desconhecida.,Desconhecido.,[Biohazard / Resident Evil 6 (2012)],https://www.residentevildatabase.com/adam-benf...,adam benford,NaN
2,Desconhecido.,Desconhecido.,Desconhecida.,Desconhecido.,"[Biohazard / Resident Evil 7 (2017), Biohazard...",https://www.residentevildatabase.com/alan-doug...,alan douglas,NaN
3,1960,O,"1,83cm (em jogos antigos); 1,90cm (em RE5)","84,5kg (em jogos antigos); 90kg (em RE5)","[Biohazard / Resident Evil Clássico (1996), Bi...",https://www.residentevildatabase.com/albert-we...,albert wesker,NaN
4,Desconhecido.,Desconhecido.,"2,90m (aprox.)",Desconhecido.,[Biohazard / Resident Evil Village (2021)],https://www.residentevildatabase.com/alcina-di...,alcina dimitrescu,NaN


In [ ]:
# um dos dados possui uma informação errada que esta quebrando no processamento
# neste caso, o personagem "Leon S. Kennedy" possui um problema na estrutura do HTML
# na informação do "Ano de nascimento", somente foi aplicado o <em> na parte "de nascimento"
df[~df['de nascimento'].isna()]

,Ano de nascimento,Tipo sanguíneo,Altura,Peso,Aparicoes,link,Nome,de nascimento
101,NaN,A,"1,78cm (em jogos antigos); 1,80cm (em RE4)","70,2kg (em jogos antigos); desconhecido (em RE4)","[Biohazard / Resident Evil 2 (1998), Biohazard...",https://www.residentevildatabase.com/leon-s-ke...,leon s kennedy,1977 (não confirmado)


In [ ]:
# exportar dados para arquivo
# 1ª opcao> CSV
# problema: lista dentro do CSV
df.to_csv('dados/dados_re.csv', index=False, sep=';')

In [ ]:
# 2ª op> melhor usar PARQUET
# se não funcionar, instalar fastparquet e pyarrow
# parquet> tipo binario que comporta diferentes tipos de dados (metadados, tipos de colunas)
df.to_parquet('dados/dados_re.parquet', index=False)

In [ ]:
# 3ª opcao> pickel
# tb é binário
# serialização de objetos
# salvar objeto do jeito que está e depois restaurar
# modelos de ML usam muito -> treinamento do algoritmo -> salvar em pkl -> usar em outro lugar (etapa de teste)
df.to_pickle('dados/dados_re.pkl')